<a href="https://colab.research.google.com/github/cabbi120101/End-to-end/blob/master/ML/2%EC%B0%A8%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8/%EB%A6%AC%EB%B7%B0_%ED%81%AC%EB%A1%A4%EB%A7%81_%EC%9D%B4%EB%8F%99%EC%9A%B1_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip install selenium

In [ ]:
import sys
if 'google.colab' in sys.modules:
    !apt-get update
    !apt install chromium-chromedriver
    !cp /usr/lib/chromium-browser/chromedriver /usr/bin
    !pip install selenium
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [ ]:
import os
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup

##############################################################  ############
##################### variable related selenium ##########################
##########################################################################
# options = webdriver.ChromeOptions()
# 크롬창(웹드라이버) 열기
driver = webdriver.Chrome('chromedriver',options=options)

# 구글 지도 접속하기
driver.get("https://www.google.com/maps/")
# driver.get('https://map.kakao.com/') 

# 검색창에 "카페" 입력하기
searchbox = driver.find_element_by_css_selector("input#searchboxinput")
searchbox.send_keys("place")

# 검색버튼 누르기
searchbutton = driver.find_element_by_css_selector("button#searchbox-searchbutton")
searchbutton.click()


cafe_name = []
comment = []
point = []

def main():
    global driver, load_wb, review_num

    driver.implicitly_wait(4)  # 렌더링 될때까지 기다린다 4초
    driver.get('https://map.kakao.com/')  # 주소 가져오기

    # 검색할 목록
    place_infos = ['신림 카페 포말 카페']

    for i, place in enumerate(place_infos):
        # delay
        if i % 4 == 0 and i != 0:
            sleep(5)
        print("#####", i)
        search(place)

    driver.quit()
    print("finish")


def search(place):
    global driver

    search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]')  # 검색 창
    search_area.send_keys(place)  # 검색어 입력
    driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
    sleep(1)

    # 검색된 정보가 있는 경우에만 탐색
    # 1번 페이지 place list 읽기
    html = driver.page_source

    soup = BeautifulSoup(html, 'html.parser')
    place_lists = soup.select('.placelist > .PlaceItem') # 검색된 장소 목록

    # 검색된 첫 페이지 장소 목록 크롤링하기
    crawling(place, place_lists)
    search_area.clear()

    # 우선 더보기 클릭해서 2페이지
    try:
        driver.find_element_by_xpath('//*[@id="info.search.place.more"]').send_keys(Keys.ENTER)
        sleep(1)

        # 2~ 5페이지 읽기
        for i in range(2, 6):
            # 페이지 넘기기
            xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
            driver.find_element_by_xpath(xPath).send_keys(Keys.ENTER)
            sleep(1)

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            place_lists = soup.select('.placelist > .PlaceItem') # 장소 목록 list

            crawling(place, place_lists)

    except ElementNotInteractableException:
        print('not found')
    finally:
        search_area.clear()


def crawling(place, place_lists):
    """
    페이지 목록을 받아서 크롤링 하는 함수
    :param place: 리뷰 정보 찾을 장소이름
    """

    while_flag = False
    for i, place in enumerate(place_lists):
        # 광고에 따라서 index 조정해야함
        if i >= 3:
           i += 1

        place_name = place.select('.head_item > .tit_name > .link_name')[0].text  # place name
        place_address = place.select('.info_item > .addr > p')[0].text  # place address

        detail_page_xpath = '//*[@id="info.search.place.list"]/li[' + str(i + 1) + ']/div[5]/div[4]/a[1]'
        driver.find_element_by_xpath(detail_page_xpath).send_keys(Keys.ENTER)
        driver.switch_to.window(driver.window_handles[-1])  # 상세정보 탭으로 변환
        sleep(1)

        print('####', place_name)

        # 첫 페이지
        extract_review(place_name)

        # 2-5 페이지
        idx = 3
        try:
            page_num = len(driver.find_elements_by_class_name('link_page')) # 페이지 수 찾기
            for i in range(page_num-1):
                # css selector를 이용해 페이지 버튼 누르기
                driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                sleep(1)
                extract_review(place_name)
                idx += 1
            driver.find_element_by_link_text('다음').send_keys(Keys.ENTER) # 5페이지가 넘는 경우 다음 버튼 누르기
            sleep(1)
            extract_review(place_name) # 리뷰 추출
        except (NoSuchElementException, ElementNotInteractableException):
            print("no review in crawling")

        # 그 이후 페이지
        while True:
            idx = 4
            try:
                page_num = len(driver.find_elements_by_class_name('link_page'))
                for i in range(page_num-1):
                    driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                    sleep(1)
                    extract_review(place_name)
                    idx += 1
                driver.find_element_by_link_text('다음').send_keys(Keys.ENTER) # 10페이지 이상으로 넘어가기 위한 다음 버튼 클릭
                sleep(1)
                extract_review(place_name) # 리뷰 추출
            except (NoSuchElementException, ElementNotInteractableException):
                print("no review in crawling")
                break

        driver.close()
        driver.switch_to.window(driver.window_handles[0])  # 검색 탭으로 전환


def extract_review(place_name):
    global driver

    ret = True

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 첫 페이지 리뷰 목록 찾기
    review_lists = soup.select('.list_evaluation > li')

    # 리뷰가 있는 경우
    if len(review_lists) != 0:
        for i, review in enumerate(review_lists):
            comment = review.select('.txt_comment > span') # 리뷰
            rating = review.select('.grade_star > em') # 별점
            val = ''
            if len(comment) != 0:
                if len(rating) != 0:
                    val = comment[0].text + '/' + rating[0].text.replace('점', '')
                else:
                    val = comment[0].text + '/0'
                    
                comment.append(comment[0].text)
                print(val)

    else:
        print('no review in extract')
        ret = False

    return ret


if __name__ == "__main__":
    main()

##### 0
#### 포말커피
지옥신림의 마지막 피난처/5
극장 의자라니 신기해. 냉침 밀크티를 바나나빈 얼음과 함께 주신다. 맛있게 마시고 막판에 얼음 으깨서 쉐이크로 먹었다. 조명과 음악까지 완벽./5
솔까 여기 카페 개지림, 개좋음 강추/5
아메리카노랑 아인슈페너 둘 다 맛있었어요. 아인슈페너는 가격이 좀 많이 비싸네요. 실내 분위기 너무 좋아요. 입구쪽에 데코가 몰려있어서 안쪽 자리에는 상대적으로 데코가 없어서 밋밋한 공간으로 보이네요./4
아니 사장님!! 왜!! 커피도 잘하시고 베이킹도 잘하세요?! 반칙반칙 ;) 말차쿠키 넘나 굿굿/5
맛있어요!!!!!친절합니다./5
카페라떼 말차쿠키 맛나요 카페분위기도좋고 직원분들 친절하세요/5
카페라떼도 맛있고 말차 스콘도 맛있네요! 카페 분위기도 좋숩니다/5
/5
/5
나만 알고 싶었는데 이젠 너무 유명해졌..... 디저트도 너무 맛있어요!/5
새로나온 디저트들도 맛있고 커피도 맛있고 신림가면 꼭 가는 카페/5
신림 카페 중 제일  디저트 맛있음 좌석 수가 적은건 아쉽/5
구조가 특이하고 음료맛은 무난/4
친절하시고 맛있음 /5
/5
세번갔는데 갈때마다 플랫화이트 마시는데 커피맛이 달라요 ㅠㅠ/4
커피맛도 너무좋고 사장님도 친절하세요 숨은 아지트입니다/5
서비스 태도 불량/1
분위기 좋아요. 커피도 맛있다는/5
20대에서 30대 여성분들 취향을 잘 아시는 것 같아여 자리가 널널한 편은 아니지만 커피도 괜찮고 사장님도 착하셨어요 노래 취향도 너무 좋았네욤 헤헤/5
/5
아주 친절하시고 커피도 맛있어요:)  조금 시끄러워요! 흡음이 안되나봐요ㅜㅜ  /4
분위기 굿/5
포말커피 짱맛... 테이크아웃하면 천원 할인해 줍니다. 지금도 참 좋지만 커피와 잘 어울리는 디저트 종류가 몇 가지 더 생겼으면 좋겠어요./5
/5
포말커피로 놀로오세요^^/5
no review in crawling
not found
finish


In [ ]:
result

['지옥신림의 마지막 피난처',
 '극장 의자라니 신기해. 냉침 밀크티를 바나나빈 얼음과 함께 주신다. 맛있게 마시고 막판에 얼음 으깨서 쉐이크로 먹었다. 조명과 음악까지 완벽.',
 '솔까 여기 카페 개지림, 개좋음 강추',
 '아메리카노랑 아인슈페너 둘 다 맛있었어요. 아인슈페너는 가격이 좀 많이 비싸네요. 실내 분위기 너무 좋아요. 입구쪽에 데코가 몰려있어서 안쪽 자리에는 상대적으로 데코가 없어서 밋밋한 공간으로 보이네요.',
 '아니 사장님!! 왜!! 커피도 잘하시고 베이킹도 잘하세요?! 반칙반칙 ;) 말차쿠키 넘나 굿굿',
 '맛있어요!!!!!친절합니다.',
 '카페라떼 말차쿠키 맛나요 카페분위기도좋고 직원분들 친절하세요',
 '카페라떼도 맛있고 말차 스콘도 맛있네요! 카페 분위기도 좋숩니다',
 '',
 '',
 '나만 알고 싶었는데 이젠 너무 유명해졌..... 디저트도 너무 맛있어요!',
 '새로나온 디저트들도 맛있고 커피도 맛있고 신림가면 꼭 가는 카페',
 '신림 카페 중 제일  디저트 맛있음 좌석 수가 적은건 아쉽',
 '구조가 특이하고 음료맛은 무난',
 '친절하시고 맛있음 ',
 '',
 '세번갔는데 갈때마다 플랫화이트 마시는데 커피맛이 달라요 ㅠㅠ',
 '커피맛도 너무좋고 사장님도 친절하세요 숨은 아지트입니다',
 '서비스 태도 불량',
 '분위기 좋아요. 커피도 맛있다는',
 '20대에서 30대 여성분들 취향을 잘 아시는 것 같아여 자리가 널널한 편은 아니지만 커피도 괜찮고 사장님도 착하셨어요 노래 취향도 너무 좋았네욤 헤헤',
 '',
 '아주 친절하시고 커피도 맛있어요:)  조금 시끄러워요! 흡음이 안되나봐요ㅜㅜ  ',
 '분위기 굿',
 '포말커피 짱맛... 테이크아웃하면 천원 할인해 줍니다. 지금도 참 좋지만 커피와 잘 어울리는 디저트 종류가 몇 가지 더 생겼으면 좋겠어요.',
 '',
 '포말커피로 놀로오세요^^']

지옥신림의 마지막 피난처
극장 의자라니 신기해. 냉침 밀크티를 바나나빈 얼음과 함께 주신다. 맛있게 마시고 막판에 얼음 으깨서 쉐이크로 먹었다. 조명과 음악까지 완벽.
솔까 여기 카페 개지림, 개좋음 강추
아메리카노랑 아인슈페너 둘 다 맛있었어요. 아인슈페너는 가격이 좀 많이 비싸네요. 실내 분위기 너무 좋아요. 입구쪽에 데코가 몰려있어서 안쪽 자리에는 상대적으로 데코가 없어서 밋밋한 공간으로 보이네요.
아니 사장님!! 왜!! 커피도 잘하시고 베이킹도 잘하세요?! 반칙반칙 ;) 말차쿠키 넘나 굿굿
맛있어요!!!!!친절합니다.
카페라떼 말차쿠키 맛나요 카페분위기도좋고 직원분들 친절하세요
카페라떼도 맛있고 말차 스콘도 맛있네요! 카페 분위기도 좋숩니다


나만 알고 싶었는데 이젠 너무 유명해졌..... 디저트도 너무 맛있어요!
새로나온 디저트들도 맛있고 커피도 맛있고 신림가면 꼭 가는 카페
신림 카페 중 제일  디저트 맛있음 좌석 수가 적은건 아쉽
구조가 특이하고 음료맛은 무난
친절하시고 맛있음 

세번갔는데 갈때마다 플랫화이트 마시는데 커피맛이 달라요 ㅠㅠ
커피맛도 너무좋고 사장님도 친절하세요 숨은 아지트입니다
서비스 태도 불량
분위기 좋아요. 커피도 맛있다는
20대에서 30대 여성분들 취향을 잘 아시는 것 같아여 자리가 널널한 편은 아니지만 커피도 괜찮고 사장님도 착하셨어요 노래 취향도 너무 좋았네욤 헤헤

아주 친절하시고 커피도 맛있어요:)  조금 시끄러워요! 흡음이 안되나봐요ㅜㅜ  
분위기 굿
포말커피 짱맛... 테이크아웃하면 천원 할인해 줍니다. 지금도 참 좋지만 커피와 잘 어울리는 디저트 종류가 몇 가지 더 생겼으면 좋겠어요.

포말커피로 놀로오세요^^


['지옥신림의 마지막 피난처',
 '극장 의자라니 신기해. 냉침 밀크티를 바나나빈 얼음과 함께 주신다. 맛있게 마시고 막판에 얼음 으깨서 쉐이크로 먹었다. 조명과 음악까지 완벽.',
 '솔까 여기 카페 개지림, 개좋음 강추',
 '아메리카노랑 아인슈페너 둘 다 맛있었어요. 아인슈페너는 가격이 좀 많이 비싸네요. 실내 분위기 너무 좋아요. 입구쪽에 데코가 몰려있어서 안쪽 자리에는 상대적으로 데코가 없어서 밋밋한 공간으로 보이네요.',
 '아니 사장님!! 왜!! 커피도 잘하시고 베이킹도 잘하세요?! 반칙반칙 ;) 말차쿠키 넘나 굿굿',
 '맛있어요!!!!!친절합니다.',
 '카페라떼 말차쿠키 맛나요 카페분위기도좋고 직원분들 친절하세요',
 '카페라떼도 맛있고 말차 스콘도 맛있네요! 카페 분위기도 좋숩니다',
 '',
 '',
 '나만 알고 싶었는데 이젠 너무 유명해졌..... 디저트도 너무 맛있어요!',
 '새로나온 디저트들도 맛있고 커피도 맛있고 신림가면 꼭 가는 카페',
 '신림 카페 중 제일  디저트 맛있음 좌석 수가 적은건 아쉽',
 '구조가 특이하고 음료맛은 무난',
 '친절하시고 맛있음 ',
 '',
 '세번갔는데 갈때마다 플랫화이트 마시는데 커피맛이 달라요 ㅠㅠ',
 '커피맛도 너무좋고 사장님도 친절하세요 숨은 아지트입니다',
 '서비스 태도 불량',
 '분위기 좋아요. 커피도 맛있다는',
 '20대에서 30대 여성분들 취향을 잘 아시는 것 같아여 자리가 널널한 편은 아니지만 커피도 괜찮고 사장님도 착하셨어요 노래 취향도 너무 좋았네욤 헤헤',
 '',
 '아주 친절하시고 커피도 맛있어요:)  조금 시끄러워요! 흡음이 안되나봐요ㅜㅜ  ',
 '분위기 굿',
 '포말커피 짱맛... 테이크아웃하면 천원 할인해 줍니다. 지금도 참 좋지만 커피와 잘 어울리는 디저트 종류가 몇 가지 더 생겼으면 좋겠어요.',
 '',
 '포말커피로 놀로오세요^^']